In [3]:
from google.cloud import storage

client = storage.Client()

In [4]:
bucket_name = "data-mattm-test-sbx"
bucket = client.get_bucket(bucket_name)

In [6]:
import os

In [8]:


destination_blob_name = "test_data.csv"
source_file_name = os.path.expanduser("~/test_dummy_data/gbq/test_data.csv")

blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)